In [2]:
%load_ext cython

In [3]:
%%cython

import random
import numpy as np
import collections
from mpire import WorkerPool

cdef list deck
cdef short deck_len, half_deck_len
cdef double parameters_0[3]
cdef int games_to_play

deck = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]*4  # deck of 52 cards
deck_len = len(deck)
half_deck_len = int(deck_len/2)
parameters_0 = [1, 1, 1]

cdef class Player():
    """
    A class that represents a player's basic moves
    """
    cdef list hand, cards
    cdef char card
    cdef double parameters[3], strategy[3]

    def __init__(self, parameters, cards = None):
        if cards is None:
            self.hand = [0]*52
        else:
            self.hand = cards
    
        self.strategy = [np.exp(i) for i in parameters]
        
    def get_card(self) :
        return self.hand.pop(0)

    def add_card(self, card):
        self.hand.append(card)

    def add_cards(self, cards):
        for card in cards:
            self.add_card(card)

    def get_hend_len(self):
        return len(self.get_hand())
    
    def get_hand(self) -> list:
        return self.hand
    
    def set_hend(self,cards):
        self.hand = cards
        
cdef class war3_player(Player):
    cdef char choice
    def make_decision(self, cards):
        choice = random.choices([1, 2, 3], self.strategy)[0]
        if choice == 1: 
            cards = sorted(cards)
        elif choice == 2:
            cards = sorted(cards, reverse=True)
        else:
            random.shuffle(cards)
            
        self.add_cards(cards)
        
def battle(player_1, player_2):
    cdef char player_1_card, player_2_card, winner, player_1_card_1, player_2_card_1, player_1_card_2, player_2_card_2
    cdef list cards
    
    player_1_card = player_1.get_card()
    player_2_card = player_2.get_card()
    cards = [player_1_card, player_2_card]
    if player_1_card > player_2_card:
        winner = 1
        return winner, cards
    elif player_2_card > player_1_card:
        winner = 2
        return winner, cards
    else:
        while 1:
            player_1_card_1 = player_1.get_card()
            player_2_card_1 = player_2.get_card()
            player_1_card_2 = player_1.get_card()
            player_2_card_2 = player_2.get_card()
            cards.extend([player_1_card_1, player_2_card_1,
                        player_1_card_2,player_2_card_2])
            if player_1_card_2 > player_2_card_2:
                winner = 1
                return winner, cards
            elif player_2_card_2 > player_1_card_2:
                winner = 2
                return winner, cards
            
def symulation(parameters1, parameters2, deck = deck):
    cdef char battle_winner
    cdef list cards
    cdef list shuffle_decl
    
    shuffle_decl = random.sample(deck, deck_len)
    player_1 = war3_player(parameters1)
    player_2 = war3_player(parameters2)
    player_1.set_hend(shuffle_decl[0:half_deck_len])
    player_2.set_hend(shuffle_decl[half_deck_len:deck_len])

    while 1:
        try:
            battle_winner, cards = battle(player_1, player_2)
            if battle_winner == 1:
                player_1.make_decision(cards)
            else:
                player_2.make_decision(cards)
        except:
            if player_1.get_hend_len() == 0:
                return 1
            else:
                return 2
# from Cython.Includes.libcpp cimport deque
            
player_1 = war3_player(parameters_0)
player_2 = war3_player(parameters_0)

games_to_play = 10000
x = [[player_1, player_2]]*games_to_play

def test(symulation, x):
    # with WorkerPool(n_jobs=5) as pool:
    pool = WorkerPool(n_jobs=5)
    return pool.map(symulation, x)

In [5]:
%%timeit
symulation([1,1,1], [1,1,1])

1.08 ms ± 50.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
%%timeit
test(symulation, x = [[player_1, player_2]]*100)

141 ms ± 41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
pool = WorkerPool(n_jobs=4)


In [38]:
%%timeit
pool.map(symulation, [[player_1, player_2]]*1000)

638 ms ± 24.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
test(symulation, x = [[player_1, player_2]]*10000)

7.98 s ± 498 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%%timeit
test(symulation, x)

2.32 s ± 33.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%%cython
games_to_play


Error compiling Cython file:
------------------------------------------------------------
...
games_to_play
^
------------------------------------------------------------

/home/olek/.cache/ipython/cython/_cython_magic_5aff6a90a17bf7c50495b0944d1247c8.pyx:1:0: undeclared name not builtin: games_to_play
